The purpose of this notebook is to use the APRS World website to download daily summary data 
for the Bergey turbines in our study. This level of data resolution is available prior to June 2018
for most turbines. The notebook uses the BeautifulSoup library to screen-scrape the data from the web
and saves it out as a compressed CSV named 'daily_summaries.csv.bz2' in this directory.

Author: Caleb Phillips (caleb.phillips@nrel.gov)

In [55]:
import pandas as pd
import numpy as np
import requests
import re
from bs4 import BeautifulSoup
from tqdm import tqdm

In [2]:
sites = pd.read_csv("bergey_sites.csv")
sites.head()

,APRS ID,AID,Public Site Name,Internal Site Name,State,Latitude,Longitude,Hub Height (m),Turbine,Periods with Consistent Generation Data,Met Tower,Met Tower Latitude,Met Tower Longitude,Measurement Height (m)
0,t007,A2719,Fremont,Fremont,MN,43.918622,-91.899498,41,Bergey Excel 10,2012/01 - 2013/12 (outage starting mid-2013/11...,NaN,NaN,NaN,NaN
1,t024,A2672,Washoe,Thomas Danzinger,NV,39.331088,-119.820234,30,Bergey Excel 10,2011/01 - 2022/12,NaN,NaN,NaN,NaN
2,t028,A3479,Towamensing,Derr,PA,40.851353,-75.598395,30,Bergey Excel 10,2011/01 - 2022/12,NaN,NaN,NaN,NaN
3,t034,A2715,Lycoming,Dan Poor,NY,43.524158,-76.372290,31,Bergey Excel 10,"2013/01 - 2017/12, 2019/01 - 2021/12",Oswego,43.464,-76.511,15.0
4,t041,A2272,Rockford,Fossil Park,IA,43.047418,-92.981674,37,Bergey Excel 10,"2013/01 - 2013/12, 2015/01 - 2022/12",NaN,NaN,NaN,NaN


In [26]:
def get_months(aid):
    url = "http://mybergey.aprsworld.com/data/ps2/historical.php?station_id="+aid
    html = requests.get(url)
    soup = BeautifulSoup(html.content, "html.parser")
    input_elements = soup.find_all("input")
    months = []
    for i in input_elements:
        if i.has_attr('name') and (i['name'] != "months[]"):
            continue
        m = re.match(r'\d{4}-\d{2}',i['value'])
        if m is None:
            continue
        months.append(i['value'])
    
    return months

In [79]:
def get_data(aid,months):
    url = "http://mybergey.aprsworld.com/data/ps2/historicalMonthS.php?months%5B%5D="+"&months%5B%5D=".join(months)+"&station_id="+aid
    html = requests.get(url)
    soup = BeautifulSoup(html.content, "html.parser")
    row_elements = soup.find_all("tr")
    rows = []
    for r in row_elements:
        td = r.find_all("td")
        # skip rows that don't look like data rows
        if len(td) == 0 or td[0] is None or td[0].text.strip == "":
            continue
        m = re.match(r'\d{4}-\d{2}-\d{2}',td[0].text.strip())
        if m is None:
            continue
        date = td[0].text.strip()
        n = td[1].text.strip()
        energy_kwh = td[2].text.strip()
        power_min = td[3].text.strip()
        power_max = td[4].text.strip()
        power_avg = td[5].text.strip()
        soft_grid = td[6].text.strip()
        faults = td[7].text.strip()
        rows.append([date,n,energy_kwh,power_min,power_max,power_avg,soft_grid,faults])
    
    return rows
    
def format_data(rows):
    df = pd.DataFrame(rows,columns=["date","n","energy_kwh","power_min_w","power_max_w","power_avg_w","soft_grid","faults"])
    df["n"] = df["n"].str.replace(',','').astype('int64')
    df["date"] = df["date"].str.replace('(Today)',"",regex=False)
    df["power_max_w"] = df["power_max_w"].str.replace(',','')
    df["power_min_w"] = df["power_min_w"].str.replace(',','')
    df["power_avg_w"] = df["power_avg_w"].str.replace(',','')
    df["energy_kwh"] = df["energy_kwh"].str.replace(',','').replace("—","")
    df = df.replace("",np.nan)
    return df

In [83]:
dfs = []
for aid in tqdm(sites['AID']):
    months = get_months(aid)
    rows = get_data(aid,months)
    df = format_data(rows)
    df["AID"] = aid
    dfs.append(df)

combined_df = pd.concat(dfs)

100%|███████████████████████████████████████████| 19/19 [02:57<00:00,  9.32s/it]


In [84]:
combined_df

,date,n,energy_kwh,power_min_w,power_max_w,power_avg_w,soft_grid,faults,AID
0,2020-02-19,6422,23,-54,4780,1333,No,None,A2719
1,2020-02-19,6422,23,-54,4780,1333,No,None,A2719
2,2020-02-18,8658,40,-49,11629,1735,No,None,A2719
3,2020-02-17,8657,21,-38,6625,928,No,None,A2719
4,2020-02-16,8659,1,-62,1668,75,No,None,A2719
...,...,...,...,...,...,...,...,...,...
3329,2011-02-11,8715,10,-63,6530,424,No,None,A2671
3330,2011-02-10,8717,8,-56,7727,393,No,None,A2671
3331,2011-02-09,8705,18,-38,6038,845,No,None,A2671
3332,2011-02-08,7136,13,-59,7383,709,No,None,A2671


In [85]:
combined_df.to_csv("daily_summaries.csv.bz2",index=False)